In [22]:
%pip install -q openai "arize-phoenix>=8.8.0" "arize-phoenix-otel>=0.8.0" openinference-instrumentation-openai python-dotenv duckdb "openinference-instrumentation>=0.1.21"

1181.39s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [23]:
import dotenv
dotenv.load_dotenv()

import nest_asyncio

import json
import phoenix as px
from phoenix.evals import TOOL_CALLING_PROMPT_TEMPLATE, OpenAIModel, llm_classify
from phoenix.experiments import evaluate_experiment, run_experiment
from phoenix.experiments.evaluators import create_evaluator
from phoenix.experiments.types import Example
from phoenix.trace import SpanEvaluations
from phoenix.trace.dsl import SpanQuery

nest_asyncio.apply()

In [24]:
px_client = px.Client(endpoint="https://app.phoenix.arize.com/s/michal-kalita")
eval_model = OpenAIModel(model="gpt-4o-mini")


/Users/michal/.pyenv/versions/3.12.2/lib/python3.12/site-packages/phoenix/utilities/client.py:60: UserWarning: The Phoenix server (11.8.0) and client (11.10.1) versions are mismatched and may have compatibility issues.
  warnings.warn(


In [45]:
query = (
    SpanQuery()
    .where(
        "span_kind == 'AGENT'",
    )
    .select()
    .select(question="input.value", output_messages="llm.output_messages", tool_definitions="llm.tools")
)

# The Phoenix Client can take this query and return the dataframe.
tool_calls_df = px_client.query_spans(query, project_name="mcp-client", timeout=None)

tool_calls_df.dropna(subset=["output_messages"], inplace=True)
tool_calls_df

,question,output_messages,tool_definitions
context.span_id,,,
4f1b0d9e2317dfcc,"Test 3, find instagram actors","[{""role"":""user"",""content"":""Test 3, find instag...","[{""name"":""get-actor-details"",""description"":""Re..."


In [46]:
def get_tool_calls(conversation_data):
    """
    Extract function calls from conversation data in the format:
    [{tool: "name_of_tool", input: {key: value}, output: {...}}, ...]
    
    Args:
        conversation_data (str or list): JSON string or parsed conversation data
    
    Returns:
        list: Array of function call objects with tool, input, and output
    """
    # Parse JSON if it's a string
    if isinstance(conversation_data, str):
        data = json.loads(conversation_data)
    else:
        data = conversation_data
    
    function_calls = []
    tool_calls_map = {}  # Map tool IDs to their calls for matching with results
    
    # First pass: collect tool calls
    for message in data:
        if isinstance(message.get('content'), list):
            for content_item in message['content']:
                if content_item.get('type') == 'tool_use':
                    tool_id = content_item.get('id')
                    tool_call = {
                        'tool': content_item.get('name'),
                        'input': content_item.get('input', {}),
                        # 'output': None  # Will be filled in second pass
                    }
                    tool_calls_map[tool_id] = tool_call
                    function_calls.append(tool_call)
    
    # Second pass: match tool results with tool calls
    # for message in data:
    #     if isinstance(message.get('content'), list):
    #         for content_item in message['content']:
    #             if content_item.get('type') == 'tool_result':
    #                 tool_id = content_item.get('tool_use_id')
    #                 if tool_id in tool_calls_map:
    #                     # Extract output from tool result
    #                     result_content = content_item.get('content', [])
    #                     output = {}
                        
    #                     # Parse the result content
    #                     for result_item in result_content:
    #                         if result_item.get('type') == 'text':
    #                             text = result_item.get('text', '')
                                
    #                             # Try to parse JSON from the text if it looks like JSON
    #                             if text.startswith('{') and text.endswith('}'):
    #                                 try:
    #                                     output = json.loads(text)
    #                                 except json.JSONDecodeError:
    #                                     output = {'text': text}
    #                             else:
    #                                 # If not JSON, store as text
    #                                 if 'text' not in output:
    #                                     output['text'] = text
    #                                 else:
    #                                     output['text'] += '\n' + text
                        
    #                     # Update the tool call with output
    #                     tool_calls_map[tool_id]['output'] = output
    
    return function_calls


tool_calls_df["tool_call"] = tool_calls_df["output_messages"].apply(get_tool_calls)
tool_calls_df.head()

,question,output_messages,tool_definitions,tool_call
context.span_id,,,,
4f1b0d9e2317dfcc,"Test 3, find instagram actors","[{""role"":""user"",""content"":""Test 3, find instag...","[{""name"":""get-actor-details"",""description"":""Re...","[{'tool': 'search-actors', 'input': {'search':..."


In [47]:
TOOL_CALLING_BASE_TEMPLATE = """
You are an evaluation assistant evaluating questions and tool calls to
determine whether the tool called would answer the question. The tool
calls have been generated by a separate agent, and chosen from the list of
tools provided below. It is your job to decide whether that agent chose
the right tool to call.

    [BEGIN DATA]
    ************
    [Question]: {question}
    ************
    [Tool Called]: {tool_call}
    [END DATA]

Your response must be single word, either "correct" or "incorrect",
and should not contain any text or characters aside from that word.
"incorrect" means that the chosen tool would not answer the question,
the tool includes information that is not presented in the question,
or that the tool signature includes parameter values that don't match
the formats specified in the tool signatures below.

"correct" means the correct tool call was chosen, the correct parameters
were extracted from the question, the tool call generated is runnable and correct,
and that no outside information not present in the question was used
in the generated question.

    [Tool Definitions]: {tool_definitions}
"""

tool_call_eval = llm_classify(
    data=tool_calls_df,
    template=TOOL_CALLING_BASE_TEMPLATE,
    rails=["correct", "incorrect"],
    model=eval_model,
    provide_explanation=True,
)

tool_call_eval["score"] = tool_call_eval.apply(
    lambda x: 1 if x["label"] == "correct" else 0, axis=1
)

tool_call_eval.head()

llm_classify |██████████| 1/1 (100.0%) | ⏳ 00:02<00:00 |  2.01s/it

,label,explanation,exceptions,execution_status,execution_seconds,score
context.span_id,,,,,,
4f1b0d9e2317dfcc,correct,The tool 'search-actors' is appropriate for th...,[],COMPLETED,2.006426,1


llm_classify |██████████| 1/1 (100.0%) | ⏳ 00:03<00:00 |  3.03s/it


In [49]:
px_client.log_evaluations(
    SpanEvaluations(eval_name="Tool Calling Eval", dataframe=tool_call_eval),
)